In [8]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import ConcatDataset
from dotenv import load_dotenv
import os


#use this
#https://docs.pytorch.org/tutorials/beginner/basics/data_tutorial.html
def get_datasetA():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize all images to 224x224
        transforms.ToTensor(),          # Convert to tensor
        transforms.Normalize([0.5], [0.5])  # Normalize (for grayscale) or use 3 values for RGB
    ])
    load_dotenv()
    if datasetName == "A":
        # Load the dataset
        dataset = datasets.ImageFolder(root=os.getenv("DATASET_A"), transform=transform)
    
        #changing the class names so it's just XX-X instead of ColorColor-Color (XX-X). 
        #This is so it matches B and C's folder structure
        new_class_names = []
        for folder_name in dataset.classes:
            if '(' in folder_name and ')' in folder_name:
                abbrev = folder_name.split('(')[-1].split(')')[0].strip()
            else:
                abbrev = folder_name  # fallback to original name
            new_class_names.append(abbrev)
    
        #updating the indexes so they match the new class names
        idx_to_abbrev = {i: abbrev for i, abbrev in enumerate(new_class_names)}
    
        #changing our classes and class indexes to match the new ones
        dataset.classes = new_class_names
        dataset.class_to_idx = idx_to_abbrev
    return dataset

def get_concatDatasets(datasetName):
      #B and C have a unique file structure... it's {DATE} SORTED / good / XX-X for every bird photographed that day... so we need to concatenate 

        root_dir = os.getenv("DATASET_B") if datasetName == "B" else os.getenv("DATASET_C")

        #list all subfolders ending with 'SORTED/good'
        img_dirs = []
        for folder in os.listdir(root_dir):
            sorted_path = os.path.join(root_dir, folder)
            #skips all non-folders and folders w/o SORTED in the name
            if not os.path.isdir(sorted_path) or 'SORTED' not in folder:
                continue
            good_path = os.path.join(sorted_path, 'good')
            if os.path.isdir(good_path):
                img_dirs.append(good_path)

        datasets_list = [datasets.ImageFolder(root=img_dir, transform=transform) for img_dir in img_dirs]
        #concating them all together
        dataset = ConcatDataset(datasets_list)

    return dataset
        


In [22]:

dataset = get_dataset("A")
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
for images, labels in dataloader:
    for label in labels:
        print(f'{label}: {dataset[label].classes}')
    break
    

AttributeError: 'tuple' object has no attribute 'classes'